# Eliminación de preguntas que no cumplen los criterios
Con el dataset limpio, podemos comenzar a realizar un análisis utilizando algunas técnicas de NLP para eliminar aquellas preguntas que no cumplan con los criterios del INE establecidos en el punto

El resultado de este paso serán:
- Un archivo que contenga el texto de las preguntas que no cumplen los criterios y una breve explicación de porqué no está cumpliéndolos (discurso de odio, violencia, malas palabras, etc.)

- Un archivo con las preguntas restantes (los campos serán los mismos que en el punto 3.1)

- ID del registro

- Entidad de Origen

- Edad

- Género

- Identificación con algún grupo en situación de discriminación

- Tema de la pregunta

- Texto de la pregunta

# Proceso

1. Importamos los módulos necesarios y leemos el nuevo DF

In [75]:
from pysentimiento import create_analyzer
from math import nan
from utils import save_df
from unidecode import unidecode
import pandas as pd
import numpy as np
import spacy
import nltk
import re

In [76]:
limpio_df = pd.read_csv('./out_datasets/1_base_limpia.csv')
limpio_df.isna().sum()

entidad                 0
edad                    0
genero                  0
grupo_discriminacion    0
tema                    0
pregunta                0
fecha                   0
dtype: int64

## Remover preguntas con palabras invalidas
Debido a que el análisis de odio tarda mucho (~4min) primero vamos hacer el dataset un poco mas chico

In [77]:
palabras_invalidas = pd.read_csv('./in_datasets/palabras_baneadas.csv')

# Pasamos todo el diccionario a minúsculas
palabras_invalidas['palabra'] = palabras_invalidas['palabra'].apply(lambda x: x.lower())

Hacemos una función que cheque si las preguntas contienen palabras baneadas, si es asi se agrega un diccionario con la palabra y el motivo de su baneo.

In [78]:
def has_invalid_words(pregunta: str, *, palabras_invalidas: pd.DataFrame) -> bool:
    # Solo se normaliza la pregunta (pasamos a lower case) ya que se asume que el diccionario de palabras baneadas cubre todos los edge-cases
    pregunta = pregunta.lower()
    pregunta = nltk.word_tokenize(pregunta)
    palabras_ban = list(palabras_invalidas['palabra'])
    for palabra in pregunta:
        if palabra in palabras_ban:
            index = palabras_ban.index(palabra)
            return { 'palabra': palabra, 'motivo': palabras_invalidas.loc[index, 'categoría'] }
    return nan

In [79]:
limpio_df['ban'] = limpio_df['pregunta'].apply(has_invalid_words, palabras_invalidas=palabras_invalidas)

Generamos un DF con las preguntas baneadas y procedemos a guardarlo con solo la información relevante

In [80]:
preguntas_ban = limpio_df.dropna()
preguntas_ban = preguntas_ban[['pregunta', 'ban']]
preguntas_ban['motivo'] = preguntas_ban['ban'].apply(lambda dict: dict['motivo'])
preguntas_ban['palabra'] = preguntas_ban['ban'].apply(lambda dict: dict['palabra'])
preguntas_ban = preguntas_ban.drop(['ban'], axis=1)
save_df(preguntas_ban, "2-0_preguntas_baneadas.csv", index=False)
print(f"Se banearon {preguntas_ban.shape[0]} preguntas.")

Se banearon 991 preguntas.


## Remover preguntas con discurso de odio

In [81]:
limpio_df = limpio_df[limpio_df['ban'].isna()]
limpio_df = limpio_df.drop('ban', axis=1)
limpio_df

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha
0,Ciudad de México,43,Femenino,No Aplica,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00
1,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00
3,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00
...,...,...,...,...,...,...,...
22885,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00
22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00
22887,Guanajuato,25,Masculino,No Aplica,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00
22888,Puebla,24,Masculino,No Aplica,No discriminación y grupos vulnerables,Qué apoyo se les brindará a los grupos más mar...,2024-02-20 11:41:00


Creamos el analizador de odio

In [82]:
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

/Users/yo-lo-pregunto/miniconda3/envs/NLP/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


Definimos una funcion para determinar si la pregunta tiene discurso de odio o no en base a un umbral

In [83]:
UMBRAL = 0.25
def has_hate_speech(pregunta: str) -> bool:
   analysis = hate_speech_analyzer.predict(pregunta)
   rate = sum(analysis.probas.values()) / len(analysis.probas)
   return analysis if rate >= UMBRAL else nan
    

In [84]:
limpio_df['hate_speech'] = limpio_df['pregunta'].apply(has_hate_speech)
limpio_df

,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,hate_speech
0,Ciudad de México,43,Femenino,No Aplica,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00,NaN
1,Aguascalientes,14,Masculino,No Aplica,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,NaN
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,NaN
3,México,34,Femenino,No Aplica,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00,NaN
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,NaN
...,...,...,...,...,...,...,...,...
22885,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00,NaN
22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,NaN
22887,Guanajuato,25,Masculino,No Aplica,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00,NaN
22888,Puebla,24,Masculino,No Aplica,No discriminación y grupos vulnerables,Qué apoyo se les brindará a los grupos más mar...,2024-02-20 11:41:00,NaN


Procesamiento las preguntas con discurso de odio y las guardamos junto con el motivo de su eliminación

In [85]:
hate_df = limpio_df.dropna()

Escribimos los motivos de la eliminación de la pregunta

In [86]:
hate_df = hate_df[["pregunta", "hate_speech"]]
hate_df['motivo'] = hate_df['hate_speech'].apply(lambda analysis: ", ".join(hate for hate in analysis.output))
hate_df['ponderacion'] = hate_df['hate_speech'].apply(lambda analysis: round(sum(analysis.probas.values())/len(analysis.probas), 2))
hate_df = hate_df.drop('hate_speech', axis=1)

Se guardan las preguntas eliminadas

In [87]:
save_df(hate_df, "2-1_hate_speech.csv", index=False)
print(f"Se eliminaron {hate_df.shape[0]} preguntas relacionadas al odio")

Se eliminaron 336 preguntas relacionadas al odio


Finalmente se guardan las preguntas que no tienen ningún discurso de odio ni palabras baneadas.

In [88]:
limpio_df = limpio_df[limpio_df['hate_speech'].isna()]
limpio_df = limpio_df.drop(columns=['hate_speech'])
print(f"Escribiendo {limpio_df.shape[0]} preguntas restantes")
save_df(limpio_df, "2-2_base_limpia.csv", index=False)

Escribiendo 21563 preguntas restantes


## Remover Preguntas fuera del tema

1. Acreamos la base de stop words y agregamos 'mexico' ya que es una palabra que solo agrega ruido al dataset porque es obvio que las preguntas van enfocadas a esta entidad nombrable.

In [89]:
stop_words = nltk.corpus.stopwords.words('spanish')
stop_words.append('mexico')
stop_words.append('ser') # Al parcer esta palabra se repite mucho
stop_words.append('of') # Dado que no se limpian las referencias, bibiliografias, etc, se agregan unas palabras en ingles
stop_words.append('corruption') 

nlp = spacy.load('es_core_news_md')

### Definicion de los Temas
Para esto vamos a usar paginas de Wikipedia para generar un vector con las palabras claves de un tema y en base a este vector hacer la similitud del coseno para saber que tanto se repiten o no estas palabras en una pregunta y asi poder dicernir si la pregunta esta relacionada o no con el tema.

#### Inconvenientes
Algunos inconvenientes con este metodo puede ser que si la pregunta es muy larga pero relacionada con el tema puede tener un menos score y no esta sobre el umbral. Se podria agregar una poderacional a dicional si se encuentra 'x' palabra en la pregunta pero agregaria mas procesamiento ya que se tendria que normalizar el resultado (o no?).


1. Se importan los modulos correspondientes

In [90]:
import wikipedia
from collections import defaultdict
from itertools import chain

wikipedia.set_lang('es')

2. Leemos la base de datos (Checkpoint)

In [91]:
limpio_df = pd.read_csv('./out_datasets/2-2_base_limpia.csv')

3. Se genera un diccionario con los queries que se haran a Wikipedia y que tendra el vector con las palabras que definen un tema.

> Los articulos de Wikipedia se seleccionaron manualmente.

In [92]:
temas = list(set(limpio_df['tema']))

# Se ordenan ya que un set es una operacion que no garantiza el orden y pues mas vale prevenir.
temas.sort()
wikipedia_pages = [["Corrupción (abuso de poder)", "Corrupción política"], ["Educación"], ["Discriminación", "Vulnerabilidad"], ["Salud", "Medicina"], ["Transparencia política"], ["Violencia contra la mujer"]]

# Se crea el diccionario de diccionarios con una sola llave por mientras.
wk_queries = { t: {'search': q } for t, q in zip(temas, wikipedia_pages)}
wk_queries

{'Combate a la corrupción': {'search': ['Corrupción (abuso de poder)',
   'Corrupción política']},
 'Educación': {'search': ['Educación']},
 'No discriminación y grupos vulnerables': {'search': ['Discriminación',
   'Vulnerabilidad']},
 'Salud': {'search': ['Salud', 'Medicina']},
 'Transparencia': {'search': ['Transparencia política']},
 'Violencia en contra de las mujeres': {'search': ['Violencia contra la mujer']}}

4. Se genera una funcion que nos ayudara para hace el procesamiento de los textos de Wikipedia llamada `_wk_process` (el motivo del nombre es que se puede hacer una funcion `wk_process` que tenga todas las llamadas de esta funcion `_wk_process`)

In [93]:
# Funcion para extraer el texto crudo del request a Wikipedia
def get_wk_content(search: str) -> str:
    return wikipedia.page(search).content

# Funcion auxiliar que crea una nueva llave a un diccionario de diccionarios y llama una funcion para llenar la lista creada
def _wk_process(wk: dict[dict], *, new_key: str, old_key: str, f, action: str='append') -> None:
    if action not in ['append', 'extend']:
        raise KeyError
    for tema in wk.keys():
        wk[tema][new_key] = []
        for item in wk[tema][old_key]:
            wk[tema][new_key].append(f(item)) if action == "append" else wk[tema][new_key].extend(f(item))

5. Se consiguen los textos

In [94]:
_wk_process(wk_queries, new_key='text', old_key='search', f=get_wk_content)
wk_queries

{'Combate a la corrupción': {'search': ['Corrupción (abuso de poder)',
   'Corrupción política'],
  'text': ['El término corrupción generalmente indica el mal uso por parte de un funcionario de su autoridad y los derechos que se le confían, así como la autoridad relacionada con este estado oficial, oportunidades, conexiones para beneficio personal, contrario a la ley y los principios morales. La corrupción también se llama soborno de funcionarios, el cual es típico de los estados de la mafia.\nUn signo característico de corrupción es un conflicto entre las acciones de un funcionario y los intereses de su empleador, o un conflicto entre las acciones de una persona elegida y los intereses de la sociedad. Muchos tipos de corrupción son similares al fraude cometido por un funcionario y pertenecen a la categoría de crímenes contra el poder estatal. \nCualquier funcionario con discreción puede estar sujeto a la corrupción en la distribución de cualquier recurso que no le pertenezca a su disc

6. Se definen las funciones para la normalizacion de los textos de Wikipedia y se hace el preprocesamiento.
> **TODO:** La normalizacion no elimina el apartado de refencias, bibliografia, etc... que pueden agregar mucho ruido.

> No se si esta funcion `normalize_wikipage` se pueda utilizar para normalizar las preguntas del debate.

In [95]:
def lemmatizer(text: str) -> str:
    doc = nlp(text)
    return " ".join([word.lemma_ for word in doc])

def normalize_wikipage(texto: str) -> list[str]:
    texto = texto.lower()
    #Usamos expresiones regulares para eliminar los subtítulos en la página 
    # los subtítulos están rodeados por ===
    # hay que refinar la expresión regular, algunos subtítulos no fueron eliminados
    texto = re.sub(r"=+\s.+[\s|.*\s]=+","", texto)

    #Para fines de este análisis vamos a eliminar todo lo que no sean palabras
    # la primera expresión regular busca eliminar todos los números que aparecen entre braquets (referencias)
    texto = re.sub(r"\[[0-9]+\]"," ",texto)
    texto = re.sub(r"\W"," ",texto)
    texto = re.sub(r'\d'," ",texto)
    texto = re.sub(r'\s+'," ",texto)

    # Lematizamos justo antes de remover acentos y cosas raras
    texto = lemmatizer(texto)
    
    texto = unidecode(texto)

    # Tokenize document
    tokens = nltk.word_tokenize(texto)

    # Filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return filtered_tokens

In [96]:
_wk_process(wk_queries, new_key='text_norm', old_key='text', f=normalize_wikipage)
wk_queries['Violencia en contra de las mujeres']

{'search': ['Violencia contra la mujer'],
 'text': ['La violencia contra la mujer es la que se ejerce por su condición de mujer, siendo esta «consecuencia de la discriminación que sufre tanto en leyes como en la práctica, y la persistencia de desigualdades por razones de género».[1]\u200b[2]\u200b\nEn esta violencia se presentan numerosas facetas que van desde la discriminación y el menosprecio hasta la agresión física, sexual, verbal, psicológica y el asesinato, manifestándose en diversos ámbitos de la vida social, laboral y política, entre los que se encuentran la propia familia, la escuela, las religiones, el Estado, entre otras.[3]\u200b Este tipo de violencia suele considerarse una forma de delito de odio, que se comete contra las mujeres o las niñas específicamente por el hecho de ser mujeres, y puede adoptar muchas formas.[4]\u200b \nEn 1993, en asamblea general, las Naciones Unidas (ONU) aprobaron la Declaración sobre la eliminación de la violencia contra la mujer, y en 1999, a

7. Se hace el conteo de las palabras de cada corpus atravez de una funcion auxiliar para poder reutilizar la funcion `_wk_process`.

In [97]:
MAX_WORDS = 5
def get_most_common(text: list) -> nltk.FreqDist:
    return nltk.FreqDist(text).most_common(MAX_WORDS)

In [98]:
_wk_process(wk_queries, new_key='fd', old_key='text_norm', f=get_most_common)
wk_queries['Salud']['fd']

[[('salud', 98),
  ('enfermedad', 38),
  ('fisico', 25),
  ('poder', 22),
  ('mental', 18)],
 [('medicina', 60),
  ('medico', 43),
  ('estudio', 41),
  ('enfermedad', 31),
  ('salud', 22)]]

8. Debido a que algunos temas tienen mas de un corpus hay mas de una lista con las palabras mas comun de cada corpur y es necesario juntarlos antes de sacar la palabras mas comun de la suma de los dos corpus

In [99]:
for tema in wk_queries.keys():
    wk_queries[tema]['most_common'] = [list(chain.from_iterable(wk_queries[tema]['fd']))]
wk_queries['Salud']['most_common']

[[('salud', 98),
  ('enfermedad', 38),
  ('fisico', 25),
  ('poder', 22),
  ('mental', 18),
  ('medicina', 60),
  ('medico', 43),
  ('estudio', 41),
  ('enfermedad', 31),
  ('salud', 22)]]

9. Una vez juntados se produce a conseguir las palabras mas repetidas de cada tema

In [100]:
def merge_common_words(words: list[tuple]) -> list[str]:
    d = defaultdict(int)
    for word, count in words:
        d[word] += count
    sorted_words = sorted(d.items(), key=lambda item: item[1], reverse=True)
    return [word[0] for word in sorted_words[:MAX_WORDS]]

In [101]:
_wk_process(wk_queries, new_key='words', old_key='most_common', f=merge_common_words, action='extend')
for tema in wk_queries.keys():
    print(wk_queries[tema]['words'])

['corrupcion', 'poder', 'pais', 'publico', 'mas']
['educacion', 'educativo', 'proceso', 'tener', 'poder']
['discriminacion', 'grupo', 'persona', 'tener', 'derecho']
['salud', 'enfermedad', 'medicina', 'medico', 'estudio']
['publico', 'deber', 'informacion', 'transparencia', 'poder']
['mujer', 'violencia', 'haber', 'mas', 'poder']


10. Se normalizan las preguntas

In [102]:
def normalize_text(texto: str) -> str:
    #1. Quitamos el '¿'
    texto = texto.replace('¿', '')
    texto = texto.replace('@', 'o')

    #2. Removemos acentos
    texto = unidecode(texto)

    #3. Remover todo caracter no alfanumerico, lowercase, sin saltos de linea.
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto, re.I|re.A).lower().strip()

    texto = lemmatizer(texto)

    #4. Tokenize document
    tokens = nltk.word_tokenize(texto)

    #5. filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]

    #6. re-create document from filtered tokens
    return ' '.join(filtered_tokens)

In [103]:
preguntas = list(map(normalize_text, limpio_df['pregunta']))

11. Se pasan los temas a una lista, porque creemos que esto es mas facil...?

In [104]:
temas = list(limpio_df['tema'])

12. Se compara los texto con el tema contra las palabras que definen el tema

> Parece que la base de Wikipedia le gusta mucho la palabra **ser** no se si vale la pena agregarla como un stop word (Al final si se agrego).

In [105]:
doc = nlp(preguntas[0])
print(f"tema: {temas[0]} text: {preguntas[0]}")
tema = nlp(temas[0].lower())
doc.similarity(tema)

tema: No discriminación y grupos vulnerables text: candidato apoyariar productor nopal alcaldia milpa alto propuesta fehaciente


0.37855280339409925

In [106]:
def get_topic_def(tema: str) -> str:
    return " ".join(word for word in wk_queries[tema]['words'])

doc = nlp(preguntas[0])
print(f"tema: {temas[0]} text: {preguntas[0]}")
tema = nlp(get_topic_def(temas[0]))
doc.similarity(tema)

tema: No discriminación y grupos vulnerables text: candidato apoyariar productor nopal alcaldia milpa alto propuesta fehaciente


0.4813423765039561

In [107]:
UMBRAL = 0.5
TOPIC_DEFINITION = True

unrelated_topics = {
    'pregunta': [],
    'tema': [],
    'score': [],
    'index': [],
}
for index, query in enumerate(zip(preguntas, temas)):
    texto, tema = query
    doc = nlp(texto)
    t = get_topic_def(tema) if TOPIC_DEFINITION else tema.lower()
    t = nlp(t)
    similarity = doc.similarity(t)
    if similarity < UMBRAL:
        unrelated_topics['pregunta'].append(limpio_df.iloc[index]['pregunta'])
        unrelated_topics['tema'].append(limpio_df.iloc[index]['tema'])
        unrelated_topics['score'].append(similarity)
        unrelated_topics['index'].append(index)

/var/folders/0r/88m3pcd926n_r_sdw7yh8wy00000gn/T/ipykernel_91822/2588300209.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc.similarity(t)


13. Se guardan las preguntas que no son validas. Al parecer este metodo guarda mas, no se si vale la pena lematizar tambien la pregunta

In [108]:
df = pd.DataFrame.from_dict(unrelated_topics)
save_df(df, "2-3_unrelated_questions.csv", index=False)
print(len(unrelated_topics['pregunta']))

6136


In [113]:
limpio_df = pd.read_csv('./out_datasets/2-2_base_limpia.csv')

In [114]:
limpio_df.index = range(len(preguntas))
limpio_df = limpio_df.drop(unrelated_topics['index'])
save_df(limpio_df, "2-4_base_limpia.csv", index=False)

In [115]:
wk_queries['Combate a la corrupción']['words']

['corrupcion', 'poder', 'pais', 'publico', 'mas']